In [1]:
import numpy as np
import warnings
import sys
import pandas as pd
import scipy
from scipy.io import arff
if not sys.warnoptions:
    warnings.simplefilter("ignore")


In [2]:
data_train, meta = scipy.io.arff.loadarff('yeast-train.arff')
df_train = pd.DataFrame(data_train)
data_test, meta = scipy.io.arff.loadarff('yeast-test.arff')
df_test = pd.DataFrame(data_test)
df_test.head()

,Att1,Att2,Att3,Att4,Att5,Att6,Att7,Att8,Att9,Att10,...,Class5,Class6,Class7,Class8,Class9,Class10,Class11,Class12,Class13,Class14
0,0.004168,-0.170975,-0.156748,-0.142151,0.058781,0.026851,0.197719,0.041850,0.066938,-0.056617,...,b'0',b'0',b'1',b'1',b'0',b'0',b'0',b'1',b'1',b'0'
1,-0.103956,0.011879,-0.098986,-0.054501,-0.007970,0.049113,-0.030580,-0.077933,-0.080529,-0.016267,...,b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0'
2,0.509949,0.401709,0.293799,0.087714,0.011686,-0.006411,-0.006255,0.013646,-0.040666,-0.024447,...,b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'1',b'1',b'0'
3,0.119092,0.004412,-0.002262,0.072254,0.044512,-0.051467,0.074686,-0.007670,0.079438,0.062184,...,b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0'
4,0.042037,0.007054,-0.069483,0.081015,-0.048207,0.089446,-0.004947,0.064456,-0.133387,0.068878,...,b'1',b'1',b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0'


In [3]:
X_train = df_train.iloc[:,:-14].values
y_train = df_train.iloc[:,-14:].values.astype(int)
X_test = df_test.iloc[:,:-14].values
y_test = df_test.iloc[:,-14:].values.astype(int)

In [4]:
print("Train_X: ",X_train.shape)
print("Train_Y: ",y_train.shape)
print("Test_X: ",X_test.shape)
print("Test_Y: ",y_test.shape)

Train_X:  (1500, 103)
Train_Y:  (1500, 14)
Test_X:  (917, 103)
Test_Y:  (917, 14)


In [5]:
def hamming_accuracy_score(y_true, y_pred, normalize=True, sample_weight=None):
    '''
    Compute the Hamming score (a.k.a. label-based accuracy) for the multi-label case
    http://stackoverflow.com/q/32239577/395857
    '''
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        #print('\nset_true: {0}'.format(set_true))
        #print('set_pred: {0}'.format(set_pred))
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/float( len(set_true.union(set_pred)) )
        #print('tmp_a: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
    return np.mean(acc_list)

## Chain Model type 1


In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.backend.tensorflow_backend import clear_session

Using TensorFlow backend.


In [7]:
def create_c1node(X_feed,y_now):
    '''
    C1 node Architecture:
    attribute:512:256:1 [saperate for each class]
    loss: Binary crossentropy
    '''
    model = Sequential()
    model.add(Dense(512,activation='relu',input_shape=(X_feed.shape[1],),kernel_initializer='glorot_uniform',name='first'))
    model.add(Dropout(0.4))
    model.add(Dense(256,activation='relu',kernel_initializer='glorot_uniform',name='nretrain1'))
    model.add(Dropout(0.4))
    model.add(Dense(128,activation='relu',kernel_initializer='glorot_uniform',name='nretrain2'))
    model.add(Dropout(0.4))
    model.add(Dense(64,activation='relu',kernel_initializer='glorot_uniform',name='last'))
    model.add(Dropout(0.4))    
    model.add(Dense(y_now.shape[1],activation='sigmoid',kernel_initializer='glorot_uniform',name='output'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    return model

In [11]:
from time import time

In [13]:
#Model chain
chain = []

#Training
X_feed = X_train.copy()
t1 = time()
for i in range(y_train.shape[1]):
    print("Training chain node ",i)
    y_now = y_train[:,[i,]].copy()
    print("Shapes:\n X = {} \n Y = {}".format(X_feed.shape,y_now.shape))

    node = create_c1node(X_feed,y_now)
    node.fit(X_feed,y_now,epochs=10,batch_size=50)
    print("Training of node {} complete\n\n".format(i))  
    #Checking accuracy of the node
    #---
    #Adding node to chain 
    chain.append(node)
    #Updating X_feed
    X_feed = np.append(X_feed,y_now,axis=1)
t2 = time()

print("Time Taken: ",t2-t1)

Training chain node  0
Shapes:
 X = (1500, 103) 
 Y = (1500, 1)
Epoch 1/10
1500/1500 [==============================] - 34s 22ms/step - loss: 0.6272 - acc: 0.6707
Epoch 2/10
1500/1500 [==============================] - 0s 130us/step - loss: 0.5536 - acc: 0.6927
Epoch 3/10
1500/1500 [==============================] - 0s 130us/step - loss: 0.5082 - acc: 0.7700
Epoch 4/10
1500/1500 [==============================] - 0s 134us/step - loss: 0.4587 - acc: 0.8067
Epoch 5/10
1500/1500 [==============================] - 0s 140us/step - loss: 0.4251 - acc: 0.8127
Epoch 6/10
1500/1500 [==============================] - 0s 131us/step - loss: 0.3896 - acc: 0.8360
Epoch 7/10
1500/1500 [==============================] - 0s 133us/step - loss: 0.3478 - acc: 0.8620
Epoch 8/10
1500/1500 [==============================] - 0s 142us/step - loss: 0.3482 - acc: 0.8593
Epoch 9/10
1500/1500 [==============================] - 0s 140us/step - loss: 0.2866 - acc: 0.8953
Epoch 10/10
1500/1500 [======================

1500/1500 [==============================] - 0s 168us/step - loss: 0.1727 - acc: 0.9427
Epoch 7/10
1500/1500 [==============================] - 0s 134us/step - loss: 0.1494 - acc: 0.9440
Epoch 8/10
1500/1500 [==============================] - 0s 137us/step - loss: 0.1220 - acc: 0.9467
Epoch 9/10
1500/1500 [==============================] - 0s 142us/step - loss: 0.1069 - acc: 0.9587
Epoch 10/10
1500/1500 [==============================] - 0s 140us/step - loss: 0.0937 - acc: 0.9587
Training of node 7 complete


Training chain node  8
Shapes:
 X = (1500, 111) 
 Y = (1500, 1)
Epoch 1/10
1500/1500 [==============================] - 1s 822us/step - loss: 0.3043 - acc: 0.9340
Epoch 2/10
1500/1500 [==============================] - 0s 134us/step - loss: 0.1904 - acc: 0.9347
Epoch 3/10
1500/1500 [==============================] - 0s 140us/step - loss: 0.1409 - acc: 0.9347
Epoch 4/10
1500/1500 [==============================] - 0s 134us/step - loss: 0.1120 - acc: 0.9347
Epoch 5/10
1500/1500 [===

In [80]:
#Model chain
y_pred = []

X_feed = X_test.copy()
i = 0
for node in chain:
    print("Getting op from chain node ",i)
    node_name = "Node" + str(i)
    print("Shapes:\n X = {}".format(X_feed.shape))
    output = node.predict(X_feed)
    output = output.round().astype(int)
    y_pred.append(output)
    X_feed = np.append(X_feed,output,axis=1)
    i+=1
    #Updating X_feed
            

Getting op from chain node  0
Shapes:
 X = (917, 103)
Getting op from chain node  1
Shapes:
 X = (917, 104)
Getting op from chain node  2
Shapes:
 X = (917, 105)
Getting op from chain node  3
Shapes:
 X = (917, 106)
Getting op from chain node  4
Shapes:
 X = (917, 107)
Getting op from chain node  5
Shapes:
 X = (917, 108)
Getting op from chain node  6
Shapes:
 X = (917, 109)
Getting op from chain node  7
Shapes:
 X = (917, 110)
Getting op from chain node  8
Shapes:
 X = (917, 111)
Getting op from chain node  9
Shapes:
 X = (917, 112)
Getting op from chain node  10
Shapes:
 X = (917, 113)
Getting op from chain node  11
Shapes:
 X = (917, 114)
Getting op from chain node  12
Shapes:
 X = (917, 115)
Getting op from chain node  13
Shapes:
 X = (917, 116)


In [63]:
predictions = np.array([0])
for i in y_pred:
    x = np.array(i)
    if predictions.shape == (1,):
        predictions = x
    else:
        predictions = np.append(predictions,x,axis=1)

In [64]:
# accuracy
from sklearn.metrics import hamming_loss, log_loss, f1_score,accuracy_score
print("Accuracy = ",accuracy_score(y_test,predictions))
print("Hamming accuracy = ",hamming_accuracy_score(y_test,predictions))
print("Hamming loss = ",hamming_loss(y_test,predictions))
print("Log loss = ",log_loss(y_test,predictions))
print("F1 score = ",f1_score(y_test,predictions,average='macro'))

Accuracy =  0.212649945474
Hamming accuracy =  0.467030446229
Hamming loss =  0.23780962766786104
Log loss =  69.8946798143
F1 score =  0.430576487903
